In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import xgboost as xgb
from xgboost import XGBClassifier

import warnings, gc

In [2]:
%%time

train = pd.read_parquet('./data/train.parquet')
label = pd.read_csv('./data/train_labels.csv')
train = train.merge(label, how='inner', on='customer_ID')

Wall time: 53.6 s


In [3]:
del label
gc.collect()

161

In [4]:
train['customer_ID'].head()

0    0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
1    0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
2    0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
3    0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
4    0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
Name: customer_ID, dtype: object

In [5]:
lab = LabelEncoder()
train['customer_ID'] = lab.fit_transform(train['customer_ID'])

In [6]:
train

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,0,0,0.0,NaN,0,0.000610,0,0
1,0,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,0,0,0.0,NaN,0,0.005492,0,0
2,0,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,0,0,0.0,NaN,0,0.006986,0,0
3,0,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,0,0,0.0,NaN,0,0.006527,0,0
4,0,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,0,0,0.0,NaN,0,0.008126,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,458912,2017-11-05,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,...,-1,-1,0,0,0.0,NaN,0,0.001498,0,0
5531447,458912,2017-12-23,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,...,-1,-1,0,0,0.0,NaN,0,0.008225,0,0
5531448,458912,2018-01-06,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,...,-1,-1,0,0,0.0,NaN,0,0.006773,0,0
5531449,458912,2018-02-06,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,...,-1,-1,0,0,0.0,NaN,0,0.001168,0,0


In [7]:
%%time

# train = train.groupby(['customer_ID']).tail(1).set_index('customer_ID')

Wall time: 0 ns


In [8]:
gc.collect()

40

In [9]:
train.shape

(5531451, 191)

In [10]:
%%time

test = pd.read_parquet('./data/test.parquet')

Wall time: 2.95 s


In [11]:
test['customer_ID'] = lab.fit_transform(test['customer_ID'])
# test = test.groupby(['customer_ID']).tail(1).set_index('customer_ID')

In [12]:
print(test.shape)
gc.collect()

(11363762, 190)


80

In [14]:
y = train.groupby(['customer_ID']).tail(1)['target']
X = train.drop(['target', 'S_2'], axis=1)
test = test.drop(['S_2'], axis=1)

X = X.fillna(-123)
test = test.fillna(-123)

In [15]:
y.value_counts()

0    340085
1    118828
Name: target, dtype: int64

In [16]:
print(X.shape, y.shape, test.shape)
gc.collect()

(5531451, 189) (458913,) (11363762, 189)


80

In [17]:
cat_cols = [
    'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126',
    'D_63', 'D_64','D_66', 'D_68'
]
num_cols = [col for col in X.columns if col not in cat_cols]

all_cols = [cat_cols, num_cols]

In [18]:
print(all_cols)

[['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], ['customer_ID', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_

In [19]:
D_n_cols = [col for col in num_cols if col.startswith('D')]
S_n_cols = [col for col in num_cols if col.startswith('S')]
P_n_cols = [col for col in num_cols if col.startswith('P')]
B_n_cols = [col for col in num_cols if col.startswith('B')]
R_n_cols = [col for col in num_cols if col.startswith('R')]
D_c_cols = [col for col in cat_cols if col.startswith('D')]
B_c_cols = [col for col in cat_cols if col.startswith('B')]

In [20]:
%%time

X_num_agg_D = X.groupby("customer_ID")[D_n_cols].agg(['mean','min', 'last'])
X_num_agg_D.columns = ['_'.join(x) for x in X_num_agg_D.columns]

X_num_agg_S = X.groupby("customer_ID")[S_n_cols].agg(['mean','min', 'last'])
X_num_agg_S.columns = ['_'.join(x) for x in X_num_agg_S.columns]

X_num_agg_P = X.groupby("customer_ID")[P_n_cols].agg(['mean','min','max' ,'last'])
X_num_agg_P.columns = ['_'.join(x) for x in X_num_agg_P.columns]

X_num_agg_B = X.groupby("customer_ID")[B_n_cols].agg(['mean','min', 'last'])
X_num_agg_B.columns = ['_'.join(x) for x in X_num_agg_B.columns]

X_num_agg_R = X.groupby("customer_ID")[R_n_cols].agg(['mean','min','last'])
X_num_agg_R.columns = ['_'.join(x) for x in X_num_agg_R.columns]

X_cat_agg_D = X.groupby("customer_ID")[D_c_cols].agg(['count','last','first','nunique'])
X_cat_agg_D.columns = ['_'.join(x) for x in X_cat_agg_D.columns]

X_cat_agg_B = X.groupby("customer_ID")[B_c_cols].agg(['count','last','nunique'])
X_cat_agg_B.columns = ['_'.join(x) for x in X_cat_agg_B.columns]

X = pd.concat([X_num_agg_D, X_num_agg_S,X_num_agg_P,X_num_agg_B,X_num_agg_R,X_cat_agg_D,X_cat_agg_B], axis=1)
del X_num_agg_D, X_num_agg_S,X_num_agg_P,X_num_agg_B,X_num_agg_R,X_cat_agg_D,X_cat_agg_B
gc.collect()

print('X shape after engineering', X.shape)

X shape after engineering (458913, 576)
Wall time: 27.9 s


In [21]:
%%time 

test_num_agg_D = test.groupby("customer_ID")[D_n_cols].agg(['mean','min', 'last'])
test_num_agg_D.columns = ['_'.join(x) for x in test_num_agg_D.columns]

test_num_agg_S = test.groupby("customer_ID")[S_n_cols].agg(['mean','min', 'last'])
test_num_agg_S.columns = ['_'.join(x) for x in test_num_agg_S.columns]

test_num_agg_P = test.groupby("customer_ID")[P_n_cols].agg(['mean','min','max', 'last'])
test_num_agg_P.columns = ['_'.join(x) for x in test_num_agg_P.columns]

test_num_agg_B = test.groupby("customer_ID")[B_n_cols].agg(['mean','min', 'last'])
test_num_agg_B.columns = ['_'.join(x) for x in test_num_agg_B.columns]

test_num_agg_R = test.groupby("customer_ID")[R_n_cols].agg(['mean','min', 'last'])
test_num_agg_R.columns = ['_'.join(x) for x in test_num_agg_R.columns]

test_cat_agg_D = test.groupby("customer_ID")[D_c_cols].agg(['count','first', 'last','nunique'])
test_cat_agg_D.columns = ['_'.join(x) for x in test_cat_agg_D.columns]

test_cat_agg_B = test.groupby("customer_ID")[B_c_cols].agg(['count','last','nunique'])
test_cat_agg_B.columns = ['_'.join(x) for x in test_cat_agg_B.columns]

test = pd.concat([test_num_agg_D, test_num_agg_S,test_num_agg_P,test_num_agg_B,test_num_agg_R,test_cat_agg_D,test_cat_agg_B], axis=1)
del test_num_agg_D, test_num_agg_S,test_num_agg_P,test_num_agg_B,test_num_agg_R,test_cat_agg_D,test_cat_agg_B
gc.collect()

print('Test shape after engineering', test.shape)

Test shape after engineering (924621, 576)
Wall time: 1min 1s


In [22]:
xgb_parms ={
    'booster': 'dart',
    'n_jobs': -1,
    'n_estimators': 1000,
    'lambda': 4.091409953463271e-08,
    'alpha': 3.6353429991712695e-08,
    'subsample': 0.6423675532438815,
    'colsample_bytree': 0.7830450413657872,
    'max_depth': 9,
    'min_child_weight': 5,
    'eta': 0.3749337530972536,
    'gamma': 0.0745370910451703,
    'grow_policy': 'depthwise',
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'rate_drop': 0.0723975209176045,
    'skip_drop': 0.9026367296518939
}

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=0
)

In [24]:
my_model = XGBClassifier(**xgb_parms)
my_model.fit(
    X_train, y_train,
    early_stopping_rounds=10,
    eval_set=[(X_valid, y_valid)],
    verbose=1
)

c:\Users\posky\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:14:57] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.48751
[1]	validation_0-logloss:0.38812
[2]	validation_0-logloss:0.33193
[3]	validation_0-logloss:0.29741
[4]	validation_0-logloss:0.27512
[5]	validation_0-logloss:0.26167
[6]	validation_0-logloss:0.25265
[7]	validation_0-logloss:0.24674
[8]	validation_0-logloss:0.24286
[9]	validation_0-logloss:0.24010
[10]	validation_0-logloss:0.23820
[11]	validation_0-logloss:0.23717
[12]	validation_0-logloss:0.23640
[13]	validation_0-logloss:0.23594
[14]	validation_0-logloss:0.23535
[15]	validation_0-logloss:0.23518
[16]	validation_0-logloss:0.23492
[17]	validation_0-logloss:0.23519
[18]	validation_0-logloss:0.23544
[19]	validation_0-logloss:0.23579
[20

XGBClassifier(alpha=3.6353429991712695e-08, base_score=0.5, booster='dart',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7830450413657872, enable_categorical=False,
              eta=0.3749337530972536, gamma=0.0745370910451703, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', lambda=4.091409953463271e-08,
              learning_rate=0.374933749, max_delta_step=0, max_depth=9,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, normalize_type='tree',
              num_parallel_tree=1, predictor='auto', random_state=0,
              rate_drop=0.0723975209176045, reg_alpha=3.63534305e-08,
              reg_lambda=4.09141023e-08, sample_type='uniform', ...)

In [25]:
pred_test = my_model.predict_proba(test)
preds = pd.DataFrame(pred_test)
pred_final = np.array(preds[1])
pred_final

array([0.04921388, 0.00154002, 0.00983813, ..., 0.80610716, 0.6682537 ,
       0.05054793], dtype=float32)

In [26]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['prediction'] = pred_final
submission

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.049214
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001540
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.009838
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.175485
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.889892
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.037112
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.829154
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.806107
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.668254


In [27]:
submission.to_csv('submission.csv', index=False)